In [ ]:
class Tokenizer_:
  def __init__(self, vocab):
    self.dict_word_to_idx = vocab
    self.dict_idx_to_word = {idx:word for word, idx in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [word.strip() for word in preprocessed if word.strip()]
    preprocessed = [word if word in self.dict_word_to_idx
                         else "<|unk|>" for word in preprocessed]
    idx_lst = [self.dict_word_to_idx[word] for word in preprocessed]
    return idx_lst

  def decode(self, idx_lst):
    text = " ".join([self.dict_idx_to_word[idx] for idx in idx_lst])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

simpleClass = Tokenizer_(vocab)

#### Dataset and Dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GPTDatasertV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []
    # tokenize the entire text
    tokens_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    # use a sliding window to chunk and book into overlapping sequences of max length
    for i in range(0, len(tokens_ids) -max_length, stride):
      input_chunk = tokens_ids[i:i+max_length]
      target_chunk = tokens_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def GPTDataloader(text, batch_size=4, max_length=256, stride=128, shuffle=True,
                    drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding('gpt2')
  dataset = GPTDatasertV1(text, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset, batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers)
  return dataloader


In [ ]:
max_length = 4
dataloader_ = GPTDataloader(
    m_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader_)
inputs, targets = next(data_iter)
inputs, inputs.shape

(tensor([[   40,   367,  2885,  1464],
         [ 1807,  3619,   402,   271],
         [10899,  2138,   257,  7026],
         [15632,   438,  2016,   257],
         [  922,  5891,  1576,   438],
         [  568,   340,   373,   645],
         [ 1049,  5975,   284,   502],
         [  284,  3285,   326,    11]]),
 torch.Size([8, 4]))

In [ ]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape, #token_embeddings

(torch.Size([8, 4, 256]),)

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer

Embedding(4, 256)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
pos_embeddings

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape, #input_embeddings

(torch.Size([8, 4, 256]),)

####Attention

In [ ]:
class SelfAttentionV2(nn.Module):
  def __init__(self, dim_in, dim_out, qkv_bias=False):
    super().__init__()
    self.W_query  = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_key    = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_value  = nn.Linear(dim_in, dim_out, bias=qkv_bias)

  def forward(self, x):
    keys = self.W_key(x)
    values = self.W_value(x)
    query = self.W_query(x)

    attention_scores = query @ keys.T
    attention_weights = torch.softmax(attention_scores/keys.shape[-1]**0.5, dim=-1)
    context_vectors = attention_weights @ values
    return context_vectors

In [ ]:
torch.manual_seed(789)
self_attention_v2 = SelfAttentionV2(dim_in=3, dim_out=2)
self_attention_v2,

(SelfAttentionV2(
   (W_query): Linear(in_features=3, out_features=2, bias=False)
   (W_key): Linear(in_features=3, out_features=2, bias=False)
   (W_value): Linear(in_features=3, out_features=2, bias=False)
 ),)

In [ ]:
self_attention_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

#### Masking additional attention with dropout

In [ ]:
''' implementation of a causal self attention and multi head attention '''

In [ ]:
class CausalAttention(nn.Module):
  def __init__(self, dim_in, dim_out, context_length,
               dropout, qkv_bias=False):
    super().__init__()
    self.dim_in = dim_in
    self.dim_out = dim_out
    self.dropout = nn.Dropout(dropout)
    self.W_query  = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_key    = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_value  = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.register_buffer('mask', torch.triu(
        torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):
    b, num_tokens, dim_in = x.shape
    keys = self.W_key(x)
    values = self.W_value(x)
    query = self.W_query(x)

    attention_scores = query @ keys.transpose(1,2)
    attention_scores = attention_scores.masked_fill(
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    attention_weights = torch.softmax(
        attention_scores/keys.shape[-1]**0.5, dim=-1)
    attention_weights = self.dropout(attention_weights)

    context_vectors = attention_weights @ values
    return context_vectors

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,
        dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
      super().__init__()
      self.heads = nn.ModuleList([CausalAttention(
                          dim_in, dim_out, context_length, dropout, qkv_bias)
                      for i in range(num_heads)])

    def forward(self, x):
      return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(dim_in, dim_out, context_length, 0.0, num_heads=1)
context_vecs = mha(batch)
mha, context_vecs, context_vecs.shape

(MultiHeadAttentionWrapper(
   (heads): ModuleList(
     (0): CausalAttention(
       (dropout): Dropout(p=0.0, inplace=False)
       (W_query): Linear(in_features=3, out_features=2, bias=False)
       (W_key): Linear(in_features=3, out_features=2, bias=False)
       (W_value): Linear(in_features=3, out_features=2, bias=False)
     )
   )
 ),
 tensor([[[-0.4519,  0.2216],
          [-0.5874,  0.0058],
          [-0.6300, -0.0632],
          [-0.5675, -0.0843],
          [-0.5526, -0.0981],
          [-0.5299, -0.1081]],
 
         [[-0.4519,  0.2216],
          [-0.5874,  0.0058],
          [-0.6300, -0.0632],
          [-0.5675, -0.0843],
          [-0.5526, -0.0981],
          [-0.5299, -0.1081]]], grad_fn=<CatBackward0>),
 torch.Size([2, 6, 2]))

#### Multi-head attention with weight splits

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,
          dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    assert(dim_out % num_heads ==0), 'dim_out must be divisible by num_heads'
    self.dim_in = dim_in
    self.dim_out = dim_out
    self.dropout = nn.Dropout(dropout)
    self.num_heads = num_heads
    self.head_dim = dim_out // num_heads

    self.W_query  = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_key    = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.W_value  = nn.Linear(dim_in, dim_out, bias=qkv_bias)
    self.out_proj = nn.Linear(dim_out, dim_out)
    self.register_buffer('mask',
        torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):
    b, num_tokens, dim_in = x.shape

    keys = self.W_key(x)
    values = self.W_value(x)
    query = self.W_query(x)

    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)
    query = query.view(b, num_tokens, self.num_heads, self.head_dim)

    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    query = query.transpose(1,2)

    attention_scores = query @ keys.transpose(2,3)
    attention_scores = attention_scores.masked_fill(
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)

    attention_weights = torch.softmax(
        attention_scores/keys.shape[-1]**0.5, dim=-1)
    attention_weights = self.dropout(attention_weights)

    context_vectors = (attention_weights @ values).transpose(1,2)
    context_vectors = context_vectors.contiguous().view(b, num_tokens, self.dim_out)
    context_vectors = self.out_proj(context_vectors)
    return context_vectors

In [ ]:
torch.manual_seed(123)
batch_size, context_length, dim_in = batch.shape
dim_out = 2

mha = MultiHeadAttention(dim_in, dim_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
mha, context_vecs, context_vecs.shape

(MultiHeadAttention(
   (dropout): Dropout(p=0.0, inplace=False)
   (W_query): Linear(in_features=3, out_features=2, bias=False)
   (W_key): Linear(in_features=3, out_features=2, bias=False)
   (W_value): Linear(in_features=3, out_features=2, bias=False)
   (out_proj): Linear(in_features=2, out_features=2, bias=True)
 ),
 tensor([[[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]],
 
         [[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 6, 2]))

In [ ]:
token_ids, tokenizer, #text_to_token_ids(token_ids, tokenizer)

(tensor([[ 6109,  3626,  6100,   345, 14504]]), <Encoding 'gpt2'>)